# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [57]:
import os
import requests, json
from dotenv import load_dotenv
import re
import zipfile
import io
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
load_dotenv()

True

In [50]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

# MODEL = "phind/phind-codellama-34b"
MODEL = "meta-llama/llama-3-70b-instruct"

# Setting up LLM

In [51]:
llm = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL)

In [52]:
def get_zip_structure(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_structure = io.StringIO()
        zip_ref.printdir(file=zip_structure)
        structure_text = zip_structure.getvalue().strip()
    return structure_text

In [ ]:
INSTRUCT_WRAP_CODE = "Wrap your code in ```python\n{CODE}\n```"
INSTRUCT_

In [60]:
input = "Convert the images in this zip file to a pdf by adding all images contained within to the PDF."


file = "images.zip"

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful coding assistant. First write a plan of execution with an outline for how to best execute the assignment, then write the code for the assignment with functional comments and debugging print statements."),
    ("user", "Assignment: {input}\nFile: {file}")
])

file += "\nZip Contents:\n" + get_zip_structure(file)

chain = prompt | llm | output_parser

output = chain.invoke({"input": input, "file": file})

python_code = re.search(r'```python(?s:(.*?))```', output).group(1)

print(python_code)


import zipfile
import img2pdf
import os
import tempfile

# Create a temporary directory to extract the zip file
with tempfile.TemporaryDirectory() as temp_dir:
    print("Extracting zip file to temporary directory...")
    with zipfile.ZipFile('images.zip', 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    
    # Get a list of image files in the temporary directory
    image_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith(('.jpg', '.jpeg'))]
    
    # Convert the list of images to a PDF
    print("Converting images to PDF...")
    pdf_bytes = img2pdf.convert(image_files)
    
    # Save the PDF to a file
    with open('output.pdf', 'wb') as f:
        f.write(pdf_bytes)
    
    print("PDF saved to output.pdf")



In [59]:
o

'Here is a plan of execution with an outline for how to best execute the assignment:\n\n**Plan of Execution**\n\nI. Import necessary libraries\n\n* `zipfile` to handle the zip file\n* `img2pdf` to convert images to PDF\n* `os` to navigate the file system\n\nII. Extract images from the zip file\n\n* Use `zipfile` to extract the contents of the zip file\n* Store the extracted images in a list\n\nIII. Convert images to PDF\n\n* Use `img2pdf` to convert each image in the list to a PDF\n* Combine the PDFs into a single PDF\n\nIV. Save the resulting PDF\n\n* Use `os` to save the resulting PDF to the file system\n\nHere is the code with functional comments and debugging print statements:\n```\nimport zipfile\nimport img2pdf\nimport os\n\n# Step I: Import necessary libraries\n\n# Step II: Extract images from the zip file\nwith zipfile.ZipFile(\'images.zip\', \'r\') as zip_ref:\n    zip_ref.extractall(\'extracted_images\')\n    print("Extracted images to \'extracted_images\' directory")\n\n# Ge

In [27]:
!pip install Pillow
!pip install reportlab
!pip install img2pdf

DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tree_of_thoughts-0.1.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tree_of_thoughts-0.1.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tree_of_thoughts-0.1.8-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/1233

In [35]:
import os
import zipfile
from PIL import Image

def convert_zip_to_pdf(input_file, output_file):
    # Create a temporary directory to extract the images
    temp_dir = "temp_images"
    os.makedirs(temp_dir, exist_ok=True)

    try:
        # Extract the images from the zipped file
        with zipfile.ZipFile(input_file, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)

        # Get the list of image files in the temporary directory
        image_files = [f for f in os.listdir(temp_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Sort the image files alphabetically to maintain the order
        image_files.sort()

        # Open the first image to get the dimensions
        with Image.open(os.path.join(temp_dir, image_files[0])) as img:
            width, height = img.size

        # Create a new PDF with the same dimensions as the images
        pdf = Image.new('RGB', (width, height))

        # Iterate over the image files and append them to the PDF
        for image_file in image_files:
            with Image.open(os.path.join(temp_dir, image_file)) as img:
                pdf.paste(img)

        # Save the PDF
        pdf.save(output_file, "PDF", resolution=100.0)

        print(f"Successfully converted {input_file} to {output_file}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

    finally:
        # Clean up the temporary directory
        for file in os.listdir(temp_dir):
            os.remove(os.path.join(temp_dir, file))
        os.rmdir(temp_dir)

# Example usage
input_file = "images.zip"
output_file = "images.pdf"
convert_zip_to_pdf(input_file, output_file)


An error occurred: list index out of range


PermissionError: [Errno 1] Operation not permitted: 'temp_images/images'